In [1]:
#Please  construct  a  new  dataset  by  either  adding  two  independent  variables  or  removing  two independent
#variables  from  finalsample.dta  dataset.  If  you  choose  to  add  two  independent variables, you could add 
#any two independent variables that you think help explain stock returns. If  you  choose  to  remove  two  
#independent  variables,  you  could  remove  any  two  independent variables that already exist in the 
#finalsample.dta dataset.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
dectreedata=pd.read_stata('/Users/jimmyaspras/Downloads/finalsample.dta')
dectreedata.columns

Index(['gvkey', 'datadate', 'sic_2', 'lagdate', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'lagdatadate', 'atq', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'dvpspq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2'],
      dtype='object')

In [3]:
dectreedata.sort_values(by=['datadate'], inplace=True)
dectreedata1=dectreedata[dectreedata['lagPrice2']>=5]#remove penny stocks
dectreedata1['Year']=dectreedata1['datadate'].dt.year
dectreedata1['Month']=dectreedata1['datadate'].dt.month
#set gvkey and datadate as the index
dectreedata1=dectreedata1.set_index(['gvkey','datadate'])
dectreedata1.head()

<ipython-input-3-b5596d44336b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dectreedata1['Year']=dectreedata1['datadate'].dt.year
<ipython-input-3-b5596d44336b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dectreedata1['Month']=dectreedata1['datadate'].dt.month


,,sic_2,lagdate,lagRet2,lagVOL2,lagPrice2,lagMV2,lagShareturnover2,lagRet2_sic,lagRet12,lagVOL12,...,BM,PE,div_p,loglagPrice2,loglagVOL12,loglagMV2,logatq,loglagVOL2,Year,Month
gvkey,datadate,,,,,,,,,,,,,,,,,,,,,
1004.0,2001-04-30,50.0,2001-02-28,-0.104674,1129800.0,13.600,366.27520,0.041950,-0.048423,0.052613,1481250.0,...,0.931715,85.000000,0.006250,2.610070,14.208397,5.903385,6.650203,13.937551,2001,4
30915.0,2001-04-30,33.0,2001-02-28,-0.050095,63300.0,10.050,91.80675,0.006929,0.004113,-0.018221,239500.0,...,1.624488,37.222222,0.000000,2.307573,12.386309,4.519686,5.489338,11.055641,2001,4
31170.0,2001-04-30,33.0,2001-02-28,-0.079365,296200.0,7.250,44.02925,0.048773,0.004113,-0.005784,345900.0,...,1.076057,34.523810,0.000000,1.981001,12.753905,3.784854,4.315018,12.598790,2001,4
31600.0,2001-04-30,33.0,2001-02-28,-0.035294,1058400.0,5.125,84.70600,0.064037,0.004113,0.078186,1158450.0,...,3.812655,36.607143,0.009756,1.634131,13.962593,4.439186,6.527946,13.872269,2001,4
61635.0,2001-04-30,33.0,2001-02-28,0.044444,201200.0,11.750,76.35150,0.030963,0.004113,0.318682,341450.0,...,1.379685,28.658537,0.000000,2.463853,12.740957,4.335348,5.667520,12.212055,2001,4


In [4]:
#Split  your  new  dataset  into  training  and  testing  samples.  Testing  sample  should  include  data 
#with year>=2016. 
#
#Drop dvpspq and atq from the train/test data
#
train=dectreedata1[dectreedata1['Year']<2016]
X_train=train[['sic_2', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2']]

In [5]:
#Set return as the dependent training variable
Y_train=train[['ret']]

In [6]:
#Set testing independent variables
test=dectreedata1[dectreedata1['Year']>=2016]
X_test=test[['sic_2', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2']]

In [7]:
#Set return as the dependent testing variable
Y_test=test[['ret']]

In [8]:
#Calculate avg monthly risk free return
rf1=pd.read_excel("/Users/jimmyaspras/Downloads/Treasury bill.xlsx")
rf1['rf']=rf1['DGS3MO']/1200
rf2=rf1[['Date','rf']].dropna()
rf2['Year']=rf2['Date'].dt.year
rf2['Month']=rf2['Date'].dt.month
rf3=rf2[['Year','Month','rf']].groupby(['Year','Month'], as_index=False).mean()

In [9]:
#Import benchmark index return
indexret1=pd.read_stata("/Users/jimmyaspras/Downloads/Index return.dta")

In [10]:
#Give a value for min_samples_leaf (you could pick any value) and train DecisionTreeRegressor using your
#new training sample. Use the trained model to predict returns based on your new testing sample. 
#Report the average return of the portfolio that consists of the 100 stocks with the highest predicted 
#returns in each year-month. Also, report the Sharpe ratio of the portfolio. 
DTree_m= DecisionTreeRegressor(min_samples_leaf=75)
DTree_m.fit(X_train,Y_train)

DecisionTreeRegressor(min_samples_leaf=75)

In [11]:
Y_predict=pd.DataFrame(DTree_m.predict(X_test), columns=['Y_predict'])
Y_test1=pd.DataFrame(Y_test).reset_index()
Comb1=pd.merge(Y_test1, Y_predict, left_index=True,right_index=True,how='inner')
Comb1['Year']=Comb1['datadate'].dt.year
Comb1['Month']=Comb1['datadate'].dt.month

In [12]:
#Rank stocks by return
rank1=Comb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
rank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
stock_long1=pd.merge(Comb1,rank1,left_index=True, right_index=True)
stock_long2=stock_long1[stock_long1['Y_predict_rank']<=100]
stock_long2['datadate'].value_counts()

2020-08-31    102
2021-02-28    101
2019-06-30    101
2016-07-31    101
2018-07-31    100
             ... 
2020-10-31     99
2020-05-31     99
2019-02-28     99
2016-05-31     99
2020-11-30     98
Name: datadate, Length: 64, dtype: int64

In [13]:
#Calculate returns
stock_long3=stock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()

In [14]:
#Merge rf and index
stock_long4=pd.merge(stock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5=pd.merge(stock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5['ret_rf']=stock_long5['ret']-stock_long5['rf']
stock_long5['ret_sp500']=stock_long5['ret']-stock_long5['sp500_ret_m']
stock_long5=sm.add_constant(stock_long5)
sm.OLS(stock_long5[['ret']],stock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 31 Jul 2021   Prob (F-statistic):                nan
Time:                        21:29:57   Log-Likelihood:                 30.645
No. Observations:                  64   AIC:                            -59.29
Df Residuals:                      63   BIC:                            -57.13
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3701      0.019     19.752      0.000       0.333       0.408
==============================================================================
Omnibus:                       31.643   Durbin-Watson:                   1.339
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               63.275
Skew:                           1.674   Prob(JB):                     1.82e-14
Kurtosis:                       6.539   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [15]:
#The decicion tree model produces returns of 37.01% above the market. This is statistically significant with a
#p-value of 0

In [16]:
#Calculate Sharpe ratio
Ret_rf=stock_long5[['ret_rf']]
SR=(Ret_rf.mean()/Ret_rf.std())*np.sqrt(12)
SR

ret_rf    8.449539
dtype: float64

In [17]:
#Give values for min_samples_leaf, n_estimators, and max_samples, and train RandomForestRegressor using your 
#new training sample. Use the trained model to predict returns based on your new testing sample. 
#Report the average return of the portfolio that consists of the 100 stocks with the highest predicted 
#returns in each year-month. Also, report the Sharpe ratio of the portfolio.
RFor_m= RandomForestRegressor(n_estimators=150, min_samples_leaf=150,bootstrap=True,max_samples=0.5,n_jobs=-1)
#n_estimators:The number of trees in the forest.
#bootstrap: whether use a different subsample of training sample to train each tree
#max_samples=0.5: randomly draw 50% of the training sample to train each tree
#n_jobs=-1 means using all CPU processors
RFor_m.fit(X_train,Y_train.values.ravel())

RandomForestRegressor(max_samples=0.5, min_samples_leaf=150, n_estimators=150,
                      n_jobs=-1)

In [18]:
#Predict returns
Y_predict=pd.DataFrame(RFor_m.predict(X_test), columns=['Y_predict'])
#Merge with actual returns
Y_test1=pd.DataFrame(Y_test).reset_index()
Comb1=pd.merge(Y_test1, Y_predict, left_index=True,right_index=True,how='inner')
Comb1['Year']=Comb1['datadate'].dt.year
Comb1['Month']=Comb1['datadate'].dt.month

In [19]:
#Rank stocks
rank1=Comb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
rank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
stock_long1=pd.merge(Comb1,rank1,left_index=True, right_index=True)
stock_long2=stock_long1[stock_long1['Y_predict_rank']<=100] #Select 100 best
stock_long2['datadate'].value_counts()

2017-10-31    100
2016-05-31    100
2019-08-31    100
2018-03-31    100
2019-04-30    100
             ... 
2020-09-30    100
2020-12-31    100
2019-03-31    100
2021-02-28    100
2016-04-30     99
Name: datadate, Length: 64, dtype: int64

In [20]:
#Calculate returns
stock_long3=stock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()

In [21]:
#Merge rf and index
stock_long4=pd.merge(stock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5=pd.merge(stock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5['ret_rf']=stock_long5['ret']-stock_long5['rf']
stock_long5['ret_sp500']=stock_long5['ret']-stock_long5['sp500_ret_m']
stock_long5=sm.add_constant(stock_long5)
sm.OLS(stock_long5[['ret']],stock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 31 Jul 2021   Prob (F-statistic):                nan
Time:                        21:41:27   Log-Likelihood:                 30.757
No. Observations:                  64   AIC:                            -59.51
Df Residuals:                      63   BIC:                            -57.36
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3699      0.019     19.774      0.000       0.332       0.407
==============================================================================
Omnibus:                       31.477   Durbin-Watson:                   1.344
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               62.599
Skew:                           1.667   Prob(JB):                     2.55e-14
Kurtosis:                       6.515   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [22]:
#The random forest decicion tree model produces returns of 36.99% above the market. This is statistically significant with a
#p-value of 0

In [23]:
#Sharpe ratio
Ret_rf=stock_long5[['ret_rf']]
SR=(Ret_rf.mean()/Ret_rf.std())*np.sqrt(12)
SR

ret_rf    8.459015
dtype: float64

In [24]:
#Give    values    for    min_samples_leaf,    n_estimators,    and    max_samples,    and    train 
#ExtraTreesRegressor  using  your  new  training  sample.  Use  the  trained  model  to  predict  returns 
#based on your new testing sample. Report the average return of the portfolio that consists of the 100 
#stocks with the highest predicted returns in each year-month. Also, report the Sharpe ratio of the portfolio.
ETree_m= ExtraTreesRegressor(n_estimators=150, min_samples_leaf=150, bootstrap=True,max_samples=0.5,n_jobs=-1)
ETree_m.fit(X_train,Y_train.values.ravel())

ExtraTreesRegressor(bootstrap=True, max_samples=0.5, min_samples_leaf=150,
                    n_estimators=150, n_jobs=-1)

In [25]:
#Predict returns
Y_predict=pd.DataFrame(ETree_m.predict(X_test), columns=['Y_predict'])
#Merge with actual
Y_test1=pd.DataFrame(Y_test).reset_index()
Comb1=pd.merge(Y_test1, Y_predict, left_index=True,right_index=True,how='inner')
Comb1['Year']=Comb1['datadate'].dt.year
Comb1['Month']=Comb1['datadate'].dt.month

In [26]:
#Rank stocks
rank1=Comb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
rank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
stock_long1=pd.merge(Comb1,rank1,left_index=True, right_index=True)
stock_long2=stock_long1[stock_long1['Y_predict_rank']<=100]
stock_long2['datadate'].value_counts()

2017-10-31    100
2016-05-31    100
2019-08-31    100
2018-03-31    100
2019-04-30    100
             ... 
2021-01-31    100
2020-09-30    100
2020-12-31    100
2019-03-31    100
2021-02-28    100
Name: datadate, Length: 64, dtype: int64

In [27]:
#Calculate returns
stock_long3=stock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()

In [28]:
#Merge rf and index
stock_long4=pd.merge(stock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5=pd.merge(stock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5['ret_rf']=stock_long5['ret']-stock_long5['rf']
stock_long5['ret_sp500']=stock_long5['ret']-stock_long5['sp500_ret_m']
stock_long5=sm.add_constant(stock_long5)
sm.OLS(stock_long5[['ret']],stock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 31 Jul 2021   Prob (F-statistic):                nan
Time:                        21:43:21   Log-Likelihood:                 30.970
No. Observations:                  64   AIC:                            -59.94
Df Residuals:                      63   BIC:                            -57.78
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3694      0.019     19.816      0.000       0.332       0.407
==============================================================================
Omnibus:                       31.257   Durbin-Watson:                   1.341
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.708
Skew:                           1.659   Prob(JB):                     3.98e-14
Kurtosis:                       6.482   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [29]:
#The extra trees regressor decicion tree model produces returns of 36.96% above the market. This is statistically significant with a
#p-value of 0

In [30]:
#Sharpe ratio
Ret_rf=stock_long5[['ret_rf']]
SR=(Ret_rf.mean()/Ret_rf.std())*np.sqrt(12)
SR

ret_rf    8.477013
dtype: float64

In [31]:
#Give  values  for  min_samples_leaf  and  max_iter,  and  train  HistGradientBoostingRegressor using your 
#new training sample. Use the trained model to predict returns based on your new testing sample. Report the average
#return of the portfolio that consists of the 100 stocks with the highest predicted returns in each year-month. 
#Also, report the Sharpe ratio of the portfolio. 
GBR_m= HistGradientBoostingRegressor(max_iter=150, min_samples_leaf=150, early_stopping='True')
#max_iter: The maximum number of iterations of the boosting process#early_stopping: If Yes, the algorithm use 
#internal cross-validation to determine max_iter 
GBR_m.fit(X_train,Y_train)

/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


HistGradientBoostingRegressor(early_stopping='True', max_iter=150,
                              min_samples_leaf=150)

In [32]:
#Predict returns
Y_predict=pd.DataFrame(GBR_m.predict(X_test), columns=['Y_predict']) 
#Merge with actual
Y_test1=pd.DataFrame(Y_test).reset_index()
Comb1=pd.merge(Y_test1, Y_predict, left_index=True,right_index=True,how='inner')
Comb1['Year']=Comb1['datadate'].dt.year
Comb1['Month']=Comb1['datadate'].dt.month

In [33]:
#Rank stocks
rank1=Comb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
rank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
stock_long1=pd.merge(Comb1,rank1,left_index=True, right_index=True)
stock_long2=stock_long1[stock_long1['Y_predict_rank']<=100]
stock_long2['datadate'].value_counts()

2019-06-30    109
2018-05-31    107
2019-12-31    103
2020-07-31    103
2020-05-31    103
             ... 
2016-04-30     95
2017-11-30     95
2018-08-31     93
2016-12-31     93
2016-02-29     92
Name: datadate, Length: 64, dtype: int64

In [34]:
#Calculate returns
stock_long3=stock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()

In [35]:
#Merge rf and index
stock_long4=pd.merge(stock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5=pd.merge(stock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5['ret_rf']=stock_long5['ret']-stock_long5['rf']
stock_long5['ret_sp500']=stock_long5['ret']-stock_long5['sp500_ret_m']
stock_long5=sm.add_constant(stock_long5)
sm.OLS(stock_long5[['ret']],stock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 31 Jul 2021   Prob (F-statistic):                nan
Time:                        21:43:30   Log-Likelihood:                 34.602
No. Observations:                  64   AIC:                            -67.20
Df Residuals:                      63   BIC:                            -65.05
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3674      0.018     20.861      0.000       0.332       0.403
==============================================================================
Omnibus:                       26.159   Durbin-Watson:                   1.304
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.314
Skew:                           1.458   Prob(JB):                     2.38e-10
Kurtosis:                       5.849   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [36]:
#The gradient boosting regressor decicion tree model produces returns of 36.72% above the market. This is statistically significant with a
#p-value of 0

In [37]:
#Sharpe ratio
Ret_rf=stock_long5[['ret_rf']]
SR=(Ret_rf.mean()/Ret_rf.std())*np.sqrt(12)
SR

ret_rf    8.922558
dtype: float64